In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(train_x, _), (test_x, _) = fashion_mnist.load_data()

train_x = train_x.astype('float32') / 255.
test_x = test_x.astype('float32') / 255.

print(train_x.shape)
print(test_x.shape)

In [ ]:
train_x = np.reshape(train_x, (len(train_x), 28, 28, 1))
test_x = np.reshape(test_x, (len(test_x), 28, 28, 1))

print(train_x.shape)
print(test_x.shape)

In [ ]:
noise = 0.5
train_x_noisy = train_x + noise * np.random.normal(loc=0.0, scale=1.0, size=train_x.shape)
test_x_noisy = test_x + noise * np.random.normal(loc=0.0, scale=1.0, size=test_x.shape)

train_x_noisy = np.clip(train_x_noisy, 0., 1.)
test_x_noisy = np.clip(test_x_noisy, 0., 1.)

In [ ]:
number_of_items = 10
plt.figure(figsize=(20, 2))

for i in range(number_of_items):
    display = plt.subplot(1, number_of_items,i+1)
    plt.imshow(test_x[i].reshape(28, 28))
    plt.gray()
    display.get_xaxis().set_visible(False)
    display.get_yaxis().set_visible(False)

In [ ]:
plt.figure(figsize=(20, 2))
for i in range(number_of_items):
    display = plt.subplot(1, number_of_items,i+1)
    plt.imshow(test_x_noisy[i].reshape(28, 28))
    plt.gray()
    display.get_xaxis().set_visible(False)
    display.get_yaxis().set_visible(False)
plt.show()

In [ ]:
input_image = Input(shape=(28, 28, 1))
print(input_image.shape)
im = Conv2D(32, (3, 3), activation='relu', padding='same')(input_image)
print(im.shape)
im = MaxPooling2D((2, 2), padding='same')(im)
print(im.shape)
im = Conv2D(32, (3, 3), activation='relu', padding='same')(im)
print(im.shape)
encoded = MaxPooling2D((2, 2), padding='same')(im)
print(encoded.shape)
# at this point the representation is (7, 7, 32)

im = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
print(im.shape)
im = UpSampling2D((2, 2))(im)
print(im.shape)
im = Conv2D(32, (3, 3), activation='relu', padding='same')(im)
print(im.shape)
im = UpSampling2D((2, 2))(im)
print(im.shape)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(im)
print(decoded.shape)

autoencoder = Model(inputs=input_image, outputs=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
autoencoder.summary()

In [ ]:
tb = [TensorBoard(log_dir='./tmp/tb', write_graph=True)]
epochs = 150 # for testing, set to 150 for actual training, best speed on GPU
batch_size = 128
autoencoder.fit(train_x_noisy, train_x,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(test_x_noisy, test_x),
                callbacks=tb)

In [ ]:
decoded_images = autoencoder.predict(test_x_noisy)
number_of_items = 10
plt.figure(figsize=(20, 2))
for item in range(number_of_items):
    display = plt.subplot(1, number_of_items,item+1)
    im = decoded_images[item].reshape(28, 28)
    plt.imshow(im)
    display.get_xaxis().set_visible(False)
    display.get_yaxis().set_visible(False)
plt.show()